In [ ]:
import numpy as np
import pandas as pd
import glob
import time
import pandas as pd
# from xml.dom import minidom


import os
import matplotlib.pyplot as plt
import keras

import cv2
import ast

In [ ]:
df=pd.read_csv('/content/drive/My Drive/json_data.csv')

In [ ]:
df.head()

,Unnamed: 0,x,y
0,0,"[[0.46875000000000006, 0.8437500000000001, 0.8...",0
1,1,"[[0.46875000000000006, 0.9062500000000001, 0.9...",1
2,2,"[[0.0703125, 0.25, 0.46875000000000006, 0.3671...",0
3,3,"[[0.0703125, 0.25, 0.46875000000000006, 0.3671...",0
4,4,"[[0.0703125, 0.25, 0.46875000000000006, 0.3671...",0


In [ ]:
df=df[df.columns[-2:]]  # Only get sentence and labels

In [ ]:
df.head()

,x,y
0,"[[0.46875000000000006, 0.8437500000000001, 0.8...",0
1,"[[0.46875000000000006, 0.9062500000000001, 0.9...",1
2,"[[0.0703125, 0.25, 0.46875000000000006, 0.3671...",0
3,"[[0.0703125, 0.25, 0.46875000000000006, 0.3671...",0
4,"[[0.0703125, 0.25, 0.46875000000000006, 0.3671...",0


In [ ]:
sentences=df['x'].values

In [ ]:
print(sentences[0])

[[0.46875000000000006, 0.8437500000000001, 0.8203125, 0.484375, 0.46875000000000006, 0.7578125, 0.25, 0.8125, 0.890625, 0.7890625, 0.796875, 0.4765625, 0.265625, 0.3671875, 0.9296875, 0.8203125, 0.8359375, 0.8203125, 0.3671875, 0.546875, 0.8203125, 0.8437500000000001, 0.7890625, 0.453125, 0.6484375, 0.8671875, 0.7734375, 0.890625, 0.7578125, 0.9062500000000001, 0.7890625, 0.8984375, 0.359375, 0.875, 0.859375, 0.8046875, 0.265625, 0.25, 0.7734375, 0.8437500000000001, 0.7578125, 0.8984375, 0.8984375, 0.4765625, 0.265625, 0.8203125, 0.8515625, 0.7578125, 0.8046875, 0.7890625, 0.265625, 0.484375, 0.46875000000000006, 0.8203125, 0.8515625, 0.8046875, 0.25, 0.7578125, 0.8437500000000001, 0.9062500000000001, 0.4765625, 0.265625, 0.6484375, 0.8671875, 0.7734375, 0.890625, 0.7578125, 0.9062500000000001, 0.7890625, 0.8984375, 0.359375, 0.875, 0.859375, 0.8046875, 0.265625, 0.25, 0.8984375, 0.890625, 0.7734375, 0.4765625, 0.265625, 0.3671875, 0.3671875, 0.9140625, 0.875, 0.8437500000000001, 0.867

In [ ]:
arr=np.zeros((len(sentences),100,100))
for i in range(len(sentences)):
   
    image=ast.literal_eval(sentences[i])
    x=np.asarray(image,dtype='float')
    arr[i]=x

    

In [ ]:
print("Input data shape : ", arr.shape)

Input data shape :  (13686, 100, 100)


In [ ]:
data = arr.reshape(arr.shape[0], 100, 100, 1)

In [ ]:
y=df['y'].values

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)

In [ ]:
# import libraries for Making Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization

In [ ]:
# A basic CNN Model
# Number of layers = 11
# Number of Convolutional layer: 3

model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0

In [ ]:
# Stop when validation accuracy > 97

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.97):
            print(" \n Reached 97% + validation accuracy")
            self.model.stop_training=True
callbacks = myCallback()

In [ ]:
batch_size = 128
num_epoch = 10
#model training
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY)
#                        callbacks=[callbacks]
                     )

Epoch 1/10
86/86 [==============================] - 4s 51ms/step - loss: 0.4072 - accuracy: 0.8339 - val_loss: 0.3332 - val_accuracy: 0.8696
Epoch 2/10
86/86 [==============================] - 4s 46ms/step - loss: 0.2227 - accuracy: 0.9235 - val_loss: 0.1530 - val_accuracy: 0.9452
Epoch 3/10
86/86 [==============================] - 4s 46ms/step - loss: 0.1332 - accuracy: 0.9573 - val_loss: 0.0990 - val_accuracy: 0.9693
Epoch 4/10
86/86 [==============================] - 4s 46ms/step - loss: 0.0673 - accuracy: 0.9792 - val_loss: 0.0708 - val_accuracy: 0.9770
Epoch 5/10
86/86 [==============================] - 4s 46ms/step - loss: 0.0545 - accuracy: 0.9824 - val_loss: 0.0641 - val_accuracy: 0.9781
Epoch 6/10
86/86 [==============================] - 4s 46ms/step - loss: 0.0406 - accuracy: 0.9879 - val_loss: 0.0518 - val_accuracy: 0.9839
Epoch 7/10
86/86 [==============================] - 4s 46ms/step - loss: 0.0283 - accuracy: 0.9904 - val_loss: 0.0464 - val_accuracy: 0.9861
Epoch 8/10
86

In [ ]:
pred=model.predict(testX)

In [ ]:
# Threshold values predicted

for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [ ]:
# Count True predicted and wrong predicted

true=0
false=0

for i in range(len(pred)):
    if pred[i] == testY[i]:
        true+=1
    else:
        false+=1
        
print("correct predicted :: ", true)
print("false prediction :: ", false)

correct predicted ::  2699
false prediction ::  39


In [ ]:
# Number of attack and benign data in test set

attack=0
benign=0
for i in range(len(testY)):
    if testY[i]==1:
        attack+=1
    else:
        benign+=1

print("Attack data in test set :: ", attack)
print(" Benign data in test set :: ", benign)

Attack data in test set ::  1478
 Benign data in test set ::  1260


In [ ]:
# Takes True positive, true negative, false positive and false negative as input
# Returns Accuracy

def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [ ]:
# Takes True Positive and false positive
# Returns Precision

def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [ ]:
#Takes True Positive and false negative
# Returns Recall

def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [ ]:
#Takes true values and predicted
# Returns Accuracy, precision and recall


def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [ ]:
accuracy,precision,recall=confusion_matrix(testY,pred)

In [ ]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 Accuracy : 0.9857560262965669 
 Precision : 0.987136086662153 
 Recall : 0.986468200270636


In [ ]:
from keras.models import save_model

In [ ]:
save_model(model, 'xsspredictmodel_new.h5')

In [ ]:
!pip install tensorflowjs 

     |████████████████████████████████| 71kB 3.2MB/s 
     |████████████████████████████████| 92kB 6.5MB/s 
     |████████████████████████████████| 137.3MB 99kB/s 
     |████████████████████████████████| 256kB 56.3MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32852 sha256=1af62043704aa0354ab541a2954b3aec910dbf1d4abe99be51e37fb3fd639331
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
Successfully built PyInquirer
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: tensorflow-hub 0.9.0
    Uninstalling tensorflow-hub-0.9.0:
      Successfully uninstalled tensorflow-hub-0.9.0


In [ ]:
!mkdir model
!tensorflowjs_converter --input_format keras xsspredictmodel_new.h5 model/

In [ ]:
!zip -r model.zip model 

  adding: model/ (stored 0%)
  adding: model/model.json (deflated 85%)
  adding: model/group1-shard2of7.bin (deflated 7%)
  adding: model/group1-shard7of7.bin (deflated 7%)
  adding: model/group1-shard5of7.bin (deflated 7%)
  adding: model/group1-shard6of7.bin (deflated 7%)
  adding: model/group1-shard3of7.bin (deflated 7%)
  adding: model/group1-shard4of7.bin (deflated 7%)
  adding: model/group1-shard1of7.bin (deflated 7%)


In [ ]:
from google.colab import files
files.download('model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>